In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set()

# Set Papermill Variables

In [5]:
input_path = "/Users/ciyer/Documents/sdsc/repos/renku-tutorial-flights-data-raw/data/v2/2019-01-flights.csv.zip"

In [3]:
input_path = "../data/flights/2019-01-flights.csv.zip"
output_path = "../data/output/2019-01-flights-cleaned.csv"

# Read in the data

In [6]:
df = pd.read_csv(input_path)

In [7]:
df.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 21
0,2019-01-01,9E,MSP,MKE,1122,1155.0,33.0,1251,1259.0,8.0,...,0.0,89.0,64.0,297.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,9E,ATL,PHF,954,1002.0,8.0,1137,1132.0,-5.0,...,0.0,103.0,90.0,508.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,9E,PHF,ATL,1213,1207.0,-6.0,1410,1408.0,-2.0,...,0.0,117.0,121.0,508.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01,9E,MSP,CLE,1005,1015.0,10.0,1302,1257.0,-5.0,...,0.0,117.0,102.0,622.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01,9E,MSP,RDU,1540,1535.0,-5.0,1921,1854.0,-27.0,...,0.0,161.0,139.0,980.0,NaN,NaN,NaN,NaN,NaN,NaN


- Look at dep time. This is a strange format. We need to parse it.

In [8]:
df.dtypes

FL_DATE                 object
OP_CARRIER              object
ORIGIN                  object
DEST                    object
CRS_DEP_TIME             int64
DEP_TIME               float64
DEP_DELAY              float64
CRS_ARR_TIME             int64
ARR_TIME               float64
ARR_DELAY              float64
CANCELLED              float64
CANCELLATION_CODE       object
DIVERTED               float64
CRS_ELAPSED_TIME       float64
ACTUAL_ELAPSED_TIME    float64
DISTANCE               float64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
Unnamed: 21            float64
dtype: object

In [9]:
# Why is DEP_TIME a float64? Is it sometimes nan?
len(df[df['DEP_TIME'].isna()])

16352

In [10]:
# How many nan times do we have if we ignore cancelled and diverted flights?
tdf = df[(df['CANCELLED'] == 0) & (df['DIVERTED'] == 0)]
len(tdf[tdf['DEP_TIME'].isna()]), len(tdf[tdf['ARR_TIME'].isna()])

(0, 0)

In [11]:
# Let's ignore the cancelled and diverted flights for the moment
df = df[(df['CANCELLED'] == 0) & (df['DIVERTED'] == 0)]

# Fix Table Format

## Convert types of time fields and drop irrelevant fields

In [12]:
# Let's convert the time fields to int64 to make conversion to time simpler and drop fields we do not care about
df = df.astype({'DEP_TIME': 'int64', 'ARR_TIME': 'int64'}).drop(['CANCELLED', 'CANCELLATION_CODE', 'DIVERTED'], 1)
df.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 21
0,2019-01-01,9E,MSP,MKE,1122,1155,33.0,1251,1259,8.0,89.0,64.0,297.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,9E,ATL,PHF,954,1002,8.0,1137,1132,-5.0,103.0,90.0,508.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,9E,PHF,ATL,1213,1207,-6.0,1410,1408,-2.0,117.0,121.0,508.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01,9E,MSP,CLE,1005,1015,10.0,1302,1257,-5.0,117.0,102.0,622.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01,9E,MSP,RDU,1540,1535,-5.0,1921,1854,-27.0,161.0,139.0,980.0,NaN,NaN,NaN,NaN,NaN,NaN


## Convert FL_DATE to datetime

In [13]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])
df.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 21
0,2019-01-01,9E,MSP,MKE,1122,1155,33.0,1251,1259,8.0,89.0,64.0,297.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,9E,ATL,PHF,954,1002,8.0,1137,1132,-5.0,103.0,90.0,508.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,9E,PHF,ATL,1213,1207,-6.0,1410,1408,-2.0,117.0,121.0,508.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01,9E,MSP,CLE,1005,1015,10.0,1302,1257,-5.0,117.0,102.0,622.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01,9E,MSP,RDU,1540,1535,-5.0,1921,1854,-27.0,161.0,139.0,980.0,NaN,NaN,NaN,NaN,NaN,NaN


# Select only flights to Austin (AUS)

In [14]:
df = df[df['DEST'] == 'AUS']

# Save the result

In [17]:
relevant_columns = ['FL_DATE', 'CRS_ARR_TIME', 'ARR_TIME']
# include ARR_DELAY if we have it
if 'ARR_DELAY' in df.columns:
    relevant_columns.extend(['ARR_DELAY'])
df[relevant_columns].head()

,FL_DATE,CRS_ARR_TIME,ARR_TIME,ARR_DELAY
238,2019-01-01,1250,1301,11.0
412,2019-01-01,1854,1854,0.0
481,2019-01-01,1503,1457,-6.0
532,2019-01-01,2259,2349,50.0
657,2019-01-01,2015,2036,21.0


In [ ]:
relevant_columns = ['FL_DATE', 'CRS_ARR_TIME', 'ARR_TIME']
# include ARR_DELAY if we have it
if 'ARR_DELAY' in df.columns:
    relevant_columns.extend(['ARR_DELAY'])
df[relevant_columns].to_csv(output_path, index=False)